In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


In [2]:
from numpy import array
from sklearn.preprocessing import Normalizer
from keras.activations import sigmoid
from keras.losses import mse
from numpy.random import uniform,normal 
from random import random

**Hyperparameters**

In [3]:
batch_size=6

**Dataset**

In [4]:
data={'Day 1':[30,40,50,20,15,60],'Day 2':[40,50,20,15,60,70],
      'Day 3':[50,20,15,60,70,50],'Target':[20,15,60,70,50,40]}

In [5]:
dataset=pd.DataFrame(data)

In [6]:
dataset

,Day 1,Day 2,Day 3,Target
0,30,40,50,20
1,40,50,20,15
2,50,20,15,60
3,20,15,60,70
4,15,60,70,50
5,60,70,50,40


In [7]:
X=dataset.drop('Target',axis=1)
y=dataset['Target']

In [8]:
X

,Day 1,Day 2,Day 3
0,30,40,50
1,40,50,20
2,50,20,15
3,20,15,60
4,15,60,70
5,60,70,50


In [9]:
X_norm = Normalizer(norm='max').fit_transform(X)

In [10]:
X_norm

array([[0.6       , 0.8       , 1.        ],
       [0.8       , 1.        , 0.4       ],
       [1.        , 0.4       , 0.3       ],
       [0.33333333, 0.25      , 1.        ],
       [0.21428571, 0.85714286, 1.        ],
       [0.85714286, 1.        , 0.71428571]])

In [11]:
array(y)

array([20, 15, 60, 70, 50, 40])

In [12]:
np.reshape(array(y),(1,-1))

array([[20, 15, 60, 70, 50, 40]])

In [13]:
y_norm=Normalizer(norm='max').fit_transform(np.reshape(array(y),(1,-1)))

In [14]:
y_norm

array([[0.28571429, 0.21428571, 0.85714286, 1.        , 0.71428571,
        0.57142857]])

**ANN From Scratch**

**Forward Propagation**


In [15]:
X_norm.shape

(6, 3)

In [ ]:
"""weight_1=array([[.2,.1],[.3,.1],[.2,.1]])
weight_2=array([[.5],[.1]])"""

'weight_1=array([[.2,.1],[.3,.1],[.2,.1]])\nweight_2=array([[.5],[.1]])'

In [16]:
weight_1=uniform(-1,1,(3,2)) #shape 3 by 2
weight_2=uniform(-1,1,(2,1))

In [17]:
weight_1

array([[ 0.19616987, -0.56750483],
       [-0.25380948, -0.89443023],
       [-0.72986039,  0.95220472]])

In [18]:
weight_2

array([[0.23229703],
       [0.45561549]])

In [20]:
X_norm[0:batch_size]#Matrix shape

array([[0.6       , 0.8       , 1.        ],
       [0.8       , 1.        , 0.4       ],
       [1.        , 0.4       , 0.3       ],
       [0.33333333, 0.25      , 1.        ],
       [0.21428571, 0.85714286, 1.        ],
       [0.85714286, 1.        , 0.71428571]])

In [23]:
X_norm[0:batch_size].shape #Matrix shape 

(6, 3)

In [24]:
output_1=X_norm[0:batch_size]@weight_1#Matrix shape multiplication
output_1

array([[-0.81520605, -0.10384236],
       [-0.38881774, -0.9675522 ],
       [-0.12431204, -0.63961551],
       [-0.72792281,  0.53942888],
       [-0.90537497,  0.06394206],
       [-0.60699273, -0.70071671]])

In [25]:
act_output=sigmoid(output_1).numpy()
act_output

array([[0.30678223, 0.47406271],
       [0.40400194, 0.27536867],
       [0.46896195, 0.34533346],
       [0.32565072, 0.63167955],
       [0.28794719, 0.51598007],
       [0.3527455 , 0.33165334]])

In [26]:
act_output.shape

(6, 2)

In [28]:
actual_output = act_output.T
actual_output.shape

(2, 6)

In [29]:
pred_output=act_output@weight_2
#pred_output=np.reshape(pred_output,(1,-1))
pred_output

array([[0.28725492],
       [0.21931068],
       [0.26627774],
       [0.36345068],
       [0.30197779],
       [0.23304813]])

In [30]:
pred_output.shape

(6, 1)

**Mean Squared Error**

In [33]:
y_true=y_norm[0][0:batch_size] #Target
y_pred=pred_output.T[0] #Its transpose
y_pred #Predicted Target

array([0.28725492, 0.21931068, 0.26627774, 0.36345068, 0.30197779,
       0.23304813])

In [34]:
y_true #Target

array([0.28571429, 0.21428571, 0.85714286, 1.        , 0.71428571,
       0.57142857])

In [36]:
error = mse(y_true,y_pred).numpy()
error

0.1731405646439288

**Backward Propagation**

In [37]:
#Error calculation
error = y_true - y_pred * (y_pred * (1 - y_pred))
error

array([0.22690185, 0.17673676, 0.80511913, 0.91591413, 0.65063266,
       0.52977432])

In [38]:
error_m = error * weight_2[1] * (actual_output[1] * (1 - actual_output[1]))
error_m

array([0.02577545, 0.01606782, 0.0829311 , 0.09709031, 0.07403388,
       0.05350268])

In [39]:
error_n = error * weight_2[0] * (actual_output[0] * (1 - actual_output[0]))
error_n

array([0.01120938, 0.0098855 , 0.04657652, 0.0467235 , 0.03098879,
       0.02809772])

**Delta Rule**

In [40]:
#Epochs calculation
epochs = 100
for ep in range(100):
  learning_rate = 0.3
  new_m = weight_2[1] + (learning_rate * error * y_pred )
  new_n = weight_2[0] + (learning_rate * error * y_pred)

In [41]:
new_m

array([0.47516909, 0.46724357, 0.51993108, 0.55548238, 0.51455848,
       0.49265437])

In [42]:
new_n

array([0.25185063, 0.2439251 , 0.29661262, 0.33216391, 0.29124001,
       0.2693359 ])